In [2]:
import numpy as np
import pandas as pd
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.stats.mstats import gmean 
import pandas_market_calendars as mcal
import QuantLib as ql
from scipy.optimize import fsolve
import calendar

# Input: S, r, sigma, (Asian) Forward curve, months associated with the curve

In [3]:
# Spot price in $/T
S = 512

# risk-free rate
r = 0.00

# implied volatility
sigma = 0.13

# Forward_curve
Forward_curve = np.array([65.60,65.90,66.12,66.38,66.82,67.13,67.33,67.40])/0.128
Months = [3,4,5,6,7,8,9,10]

# Fit the d, the effective  (storage-convenience) yield, by the forward curve, assuming that d is a piecewise constant function of term
$F_{0,T_1} = S e^{(r-d_1)T_1}$ <br>
$F_{0,T_2} = S e^{(r-d_1)T_1+(r-d_2)(T_2-T_1)}$ <br>
$F_{0,T_3} = S e^{(r-d_1)T_1+(r-d_2)(T_2-T_1)+(r-d_3)(T_3-T_2)}$ <br><br>

$Future_1$ = $\frac{\sum_{i}F_{0,i}}{n}$ for i in [T_0,T_1] <br>
$Future_2$ = $\frac{\sum_{i}F_{0,i}}{n}$ for i in [T_1,T_2] <br>

## deal with trading dates

In [4]:
ice = mcal.get_calendar('ICE')
today = datetime.today().strftime("%Y-%m-%d")


T01_list = []
#Jump_point_daycount = []
Jump_point_calendar_date = []
for i in [3,4,5,6,7,8,9]:
    calender_first_date = '2020-0'+str(i)+'-01'
    if i in [1,3,5,7,8,10,12]:
        calender_last_date = '2020-0'+str(i)+'-31'
    elif i == 2:
        calender_last_date = '2020-0'+str(i)+'-29'
    else:
        calender_last_date = '2020-0'+str(i)+'-30'

    averaging_dates = ice.schedule(calender_first_date,calender_last_date)
    trading_first_date = averaging_dates.index[0].strftime("%Y-%m-%d")
    trading_last_date =  averaging_dates.index[-1].strftime("%Y-%m-%d")
    if today == ice.schedule(today,trading_first_date).index[0].strftime("%Y-%m-%d"):
        T0 = len(ice.schedule(today,trading_first_date))-1
        T1 = len(ice.schedule(today,trading_last_date))-1
    else:
        T0 = len(ice.schedule(today,trading_first_date))
        T1 = len(ice.schedule(today,trading_last_date))
    T01_list.append([T0,T1])
    Jump_point_calendar_date.append(trading_last_date)
    
for i in [10]:
    calender_first_date = '2020-'+str(i)+'-01'
    if i in [1,3,5,7,8,10,12]:
        calender_last_date = '2020-'+str(i)+'-31'
    elif i == 2:
        calender_last_date = '2020-'+str(i)+'-29'
    else:
        calender_last_date = '2020-'+str(i)+'-30'

    averaging_dates = ice.schedule(calender_first_date,calender_last_date)
    trading_first_date = averaging_dates.index[0].strftime("%Y-%m-%d")
    trading_last_date =  averaging_dates.index[-1].strftime("%Y-%m-%d")
    T0 = len(ice.schedule(today,trading_first_date))-1
    T1 = len(ice.schedule(today,trading_last_date))-1
    T01_list.append([T0,T1])
    Jump_point_calendar_date.append(trading_last_date)


In [5]:
T01_list # starting and ending dates in number of days from today (0 stands for today)

[[5, 26],
 [27, 47],
 [48, 68],
 [69, 90],
 [91, 113],
 [114, 134],
 [135, 156],
 [157, 178]]

In [6]:
Jump_point_calendar_date # dates when d jumps

['2020-03-31',
 '2020-04-30',
 '2020-05-29',
 '2020-06-30',
 '2020-07-31',
 '2020-08-31',
 '2020-09-30',
 '2020-10-30']

##### Solve d by bootstraps

In [7]:
factor = 1
T_temp = 0
d_list =[]
factor_list = []
for i in range(len(T01_list)):
    T0,T1 =  T01_list[i]
    def F(d):
        return np.sum([S*np.exp((r - d)*i/258) for i in range(T0-T_temp,T1+1-T_temp)])/(T1-T0+1) - Forward_curve[i]/factor
    d = fsolve(F,0)[0]
    d_list.append(d)
    factor *= np.exp((r - d)*(T1-T_temp)/258)
    factor_list.append(factor)
    T_temp = T1

In [8]:
d_list # piecewise constants of d, used alongside the list of months

[-0.016245780248153378,
 -0.0914576247756574,
 0.004919370784387824,
 -0.09226617205911024,
 -0.06133826039518713,
 -0.047238223365295644,
 -0.025673042401012366,
 0.00012380613034111896]

## BBMC
inputs: K, n_path (# of paths)

In [15]:
K=600
n_path =1000000

In [13]:
def last_day_before_averaging(M):
    day = calendar.monthrange(2020,M-1)[1]
    if M<11:
        return '2020-0'+str(M-1)+'-'+str(day)
    else:
        return '2020-'+str(M-1)+'-'+str(day)

In [14]:
drift_list = []
for i in range(8):

    dividend = d_list[i]
    T0,T1 = T01_list[i]    
    drift_list+=[(r-dividend-0.5*sigma**2)/258]*(T1-T0+1)

n_dot = len(drift_list)+1 # number of N(0,1) to generated, including the last day before averaging



In [32]:
#generatiung paths
paths = np.random.normal(0,1,(n_path,n_dot))

#transform by volatility
vol = np.diag([sigma*((T01_list[0][0]-1)/258)**0.5]+[sigma*(1/258)**0.5]*(n_dot-1))
paths = paths @ vol

#transform by drift

drift = np.array([np.log(S)+(r-d_list[0]-0.5*sigma**2)* (T01_list[0][0]-1)/258]+drift_list)
paths += drift

#cumsum to get lg(S)
paths = np.cumsum(paths,axis = 1)

#transform lg(S) to S
paths = np.exp(paths)

#drop the first price which is for 29-02-2020
paths = np.delete(paths,[0],axis=1)

#payoff for each month average 
Payoff = np.zeros([n_path,len(Months)])
for i in range(len([3,4,5,6,7,8,9,10])):
#     T_start = T01_list[i][0]-T01_list[0][0]
#     T_end = T01_list[i][1]-T01_list[0][0]
#     Payoff[M] = np.average(paths[:,T_start:(T_end+1)])
    T_start = T01_list[i][0]-T01_list[0][0]
    T_end = T01_list[i][1]-T01_list[0][0]
    Payoff[:,i] = np.average(paths[:,T_start:(T_end+1)],axis=1)
    Payoff[:,i] = list(map(lambda x: max(x,0),Payoff[:,i]-K))

#price for each option
Payoff = np.average(Payoff,axis =0)*[np.exp(-r*(t+5)/258) for  t in np.array(T01_list)[:,1]]


In [33]:
Payoff

array([0.        , 0.00387972, 0.06561855, 0.27354833, 0.74994119,
       1.40870152, 2.13059791, 2.866917  ])

In [34]:
Q3 = np.sum(Payoff[[Months.index(i) for i in [7,8,9]]]) # portfolio of options

In [35]:
Q3

4.289240618110905